# TUJI1  Dataset Indoor Locanization 

This notebook contain TUJI1  Dataset analyze and  model development

In [31]:
# libraires
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import numpy as np
import tensorflow as ts
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

In [3]:
# Data reading
df_train_c=pd.read_csv("DATASET/Coordinates_training.csv",names=["x","y","z","floor","build","ticket"])
df_test_c=pd.read_csv("DATASET/Coordinates_testing.csv",names=["x","y","z","floor","build","ticket"])
df_train_rss=pd.read_csv("DATASET/RSS_training.csv",header=None)
df_test_rss=pd.read_csv("DATASET/RSS_testing.csv",header=None)
df_labels=pd.read_csv("DATASET/Device_labels.csv")

In [4]:
# EDA - Explaratory Data Analysis
df_train_c.head()

,x,y,z,floor,build,ticket
0,22.640,14.903,0,0,0,1
1,22.048,14.903,0,0,0,1
2,21.459,14.903,0,0,0,1
3,20.859,14.903,0,0,0,1
4,20.262,14.903,0,0,0,1


In [5]:
df_train_rss.head()

,0,1,2,3,4,5,6,7,8,9,...,300,301,302,303,304,305,306,307,308,309
0,-54,-57,-60,-60,-75,-75,-88,-88,-88,-88,...,100,100,100,100,100,100,100,100,100,100
1,-51,-57,-62,-62,-74,-74,-88,-88,100,-88,...,100,100,100,100,100,100,100,100,100,100
2,-59,-61,-64,-64,-74,-74,-85,-85,-88,-87,...,100,100,100,100,100,100,100,100,100,100
3,-59,-66,-64,-62,-70,-70,-85,-86,-89,-89,...,100,100,100,100,100,100,100,100,100,100
4,-58,-64,-65,-63,-68,-68,-87,-87,-82,-83,...,100,100,100,100,100,100,100,100,100,100


In [6]:
df_test_c.head()

,x,y,z,floor,build,ticket
0,9.8315,0.8950,0,0,0,1
1,9.8315,3.8750,0,0,0,1
2,9.8315,4.4720,0,0,0,1
3,9.8315,5.6640,0,0,0,1
4,9.8315,6.2615,0,0,0,1


In [7]:
df_test_rss.head()

,0,1,2,3,4,5,6,7,8,9,...,300,301,302,303,304,305,306,307,308,309
0,-72,-72,100,100,-52,-53,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1,-81,-63,-89,-89,-50,-50,-87,-88,100,100,...,100,100,100,100,100,100,100,100,100,100
2,-77,-60,-87,-86,-49,-49,-88,-88,100,100,...,100,100,100,100,100,100,100,100,100,100
3,-71,-60,-89,-89,-40,-40,100,100,-88,-87,...,100,100,100,100,100,100,100,100,100,100
4,-80,-69,-85,-85,-43,-44,-87,-87,100,-92,...,100,100,100,100,100,100,100,100,100,100


In [8]:
df_labels

,Device,Label
0,S20,1
1,S7,2
2,POCO,3
3,tabS7,4
4,A12,5


In [9]:
# Null values check 
df_test_c.isnull().sum().sum()
 
df_train_c.isnull().sum().sum()
 
df_test_rss.isnull().sum().sum()
 
df_train_rss.isnull().sum().sum()

0

In [10]:
# Replace 100 rss value with -110 [1]
df_test_rss.replace(100,-110,inplace=True)

df_train_rss.replace(100,-110,inplace=True)
"""
M. Nowicki and J. Wietrzykowski, “Low-effort place recognition with
wifi fingerprints using deep learning,” 11 2016
"""

'\nM. Nowicki and J. Wietrzykowski, “Low-effort place recognition with\nwifi fingerprints using deep learning,” 11 2016\n'

In [11]:
df_train_rss.head()

,0,1,2,3,4,5,6,7,8,9,...,300,301,302,303,304,305,306,307,308,309
0,-54,-57,-60,-60,-75,-75,-88,-88,-88,-88,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110
1,-51,-57,-62,-62,-74,-74,-88,-88,-110,-88,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110
2,-59,-61,-64,-64,-74,-74,-85,-85,-88,-87,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110
3,-59,-66,-64,-62,-70,-70,-85,-86,-89,-89,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110
4,-58,-64,-65,-63,-68,-68,-87,-87,-82,-83,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110


## X Y Location Predict Modeling

### Preprocesing

In [12]:
"""
Mustafa Eren Yildirim BALKAN JOURNAL OF ELECTRICAL & COMPUTER ENGINEERING,
Vol. 10, No. 1, January 2022
"""
df_train_x=df_train_rss.iloc[:,1:30]

df_train_y=df_train_c[["x","y"]]

In [13]:
# Undersampling for easy modeling  
data=pd.concat([df_train_x,df_train_y],axis=1)

In [14]:
# Drop Duplicates
data.drop_duplicates(inplace=True)
data.shape

(6752, 31)

In [15]:
data

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,x,y
0,-57,-60,-60,-75,-75,-88,-88,-88,-88,-86,...,-92,-92,-110,-94,-110,-110,-110,-110,22.6400,14.9030
1,-57,-62,-62,-74,-74,-88,-88,-110,-88,-82,...,-83,-110,-110,-93,-110,-110,-110,-110,22.0480,14.9030
2,-61,-64,-64,-74,-74,-85,-85,-88,-87,-89,...,-89,-91,-110,-93,-110,-110,-110,-110,21.4590,14.9030
3,-66,-64,-62,-70,-70,-85,-86,-89,-89,-80,...,-86,-110,-110,-92,-110,-110,-110,-110,20.8590,14.9030
4,-64,-65,-63,-68,-68,-87,-87,-82,-83,-86,...,-88,-89,-110,-110,-110,-110,-110,-110,20.2620,14.9030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6747,-70,-110,-110,-48,-48,-110,-92,-110,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-90,12.2185,8.6465
6748,-110,-91,-110,-53,-53,-110,-110,-81,-110,-81,...,-86,-84,-110,-87,-110,-110,-110,-110,13.4080,8.6465
6749,-72,-91,-91,-55,-55,-110,-110,-82,-82,-110,...,-110,-110,-88,-110,-110,-110,-110,-110,14.0030,8.6465
6750,-72,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-75,-110,-110,-110,-110,-110,-110,-110,14.5995,8.6465


In [16]:
# X,Y split
X=data.drop(["x","y"],axis=1)

Y=data[["x","y"]]

### Modeling

In [80]:

rf=RandomForestRegressor(n_estimators=200,random_state=42)
multi_output_rf=MultiOutputRegressor(rf,n_jobs=-1)

In [81]:
# Model Fiting
multi_output_rf.fit(X,Y)

MultiOutputRegressor(estimator=RandomForestRegressor(n_estimators=200,
                                                     random_state=42),
                     n_jobs=-1)

In [21]:
import joblib
joblib.dump(multi_output_rf, "multi_output_rf_model.pkl")

['multi_output_rf_model.pkl']

### Model Sucsess

In [82]:
# Prepare test data
df_test_x=df_test_rss.iloc[:,1:30]
df_test_y=df_test_c[["x","y"]]
data_test=pd.concat([df_test_x,df_test_y],axis=1)
data_test.drop_duplicates(inplace=True)
X_test=data_test.drop(["x","y"],axis=1)
Y_test=data_test[["x","y"]]

In [77]:
y_pred=multi_output_rf.predict(X=X_test)

In [78]:
r2_score(Y_test,y_pred)

0.8658878367724354

In [79]:
mean_absolute_error(Y_test,y_pred)

1.5658784216902877

#### ANN

In [64]:
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(X)
x_test_scaled=scaler.fit_transform(X_test)

In [65]:
#FNN
model=Sequential()

model.add(Dense(64,input_dim=29, activation="relu"))

model.add(Dense(32,activation="relu"))
model.add(Dense(32,activation="relu"))

model.add(Dense(2,activation="linear"))

/home/selman/anaconda3/envs/my_env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [66]:
model.compile(optimizer="adam",loss="mse",metrics=["mae"])

history=model.fit(x_train_scaled,Y,epochs=100,batch_size=32)

Epoch 1/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 123.8496 - mae: 8.1891
Epoch 2/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 13.6177 - mae: 2.9061
Epoch 3/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11.5662 - mae: 2.6818
Epoch 4/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.7157 - mae: 2.5636
Epoch 5/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.9935 - mae: 2.4911
Epoch 6/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.1606 - mae: 2.3886
Epoch 7/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.6950 - mae: 2.3224
Epoch 8/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.4484 - mae: 2.2877
Epoch 9/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.7949 - mae: 2.2054
Epoch 10/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.3223 - mae: 2.1394
Epoch 11/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.1802 - mae: 2.1209
Epoch 12/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.8047 - mae: 2.0457
Epoch 13

In [67]:
joblib.dump(model, "ann_model.pkl")

['ann_model.pkl']

In [71]:
y_pred_ann=model.predict(x_test_scaled)

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [72]:
r2_score(Y_test,y_pred_ann)

0.8085299730300903

In [73]:
mean_absolute_error(Y_test,y_pred_ann)

1.991505502107916

### Sources
[https://www.sciencedirect.com/science/article/pii/S2352340924003251]<br>
[https://zenodo.org/records/7641701]<br>
[https://dergipark.org.tr/en/download/article-file/1129736]<br>
[https://dergipark.org.tr/en/download/article-file/1840866]<br>
[https://dergipark.org.tr/tr/download/article-file/388471]<br>
[https://dergipark.org.tr/tr/download/article-file/2354636]